<a href="https://colab.research.google.com/github/JonaJJSJ-crypto/E-Physics/blob/main/IntercambioCalor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Simulador de Equilibrio Térmico**

## por: Jonathan Joel Sánchez Jácome MSc.

En este simulador se muestra el intercambio de calor entre dos cuerpos representados por cubos. Cada cubo tiene un deslizador que nos permite controlar la temperatura y la masa de cada uno. Esto nos permite estudiar empíricamente cuál es la temperatura en que se alcanza el equilibrio térmico.

En esta versión, ambos cuerpos tienen la misma capacidad calorífica, pero se plantea implementar esta prestación en futuras versiones. Si tienes algún inconveniente o sugerencia para este simulador, contacta a: **jjsanchezj@outlook.com**.

---

## **Modo de uso:**

Este programa está hecho en Google Colaboratory.

1. Para inicializar el programa, presiona **"Ctrl + F9"** o presiona el botón con el símbolo **(▶)**.
2. Una vez ejecutado, en tu pantalla deben aparecer dos bloques de color **azul pálido**.
3. Inicialmente, ambos bloques aparecen con una temperatura de **1 grado centígrado** y una masa de **100 gramos**.
4. Siéntete libre de modificar estas cantidades de acuerdo con la experiencia que desees realizar.
5. Para iniciar el intercambio de calor, presiona el botón |**▶ Play**| que aparece debajo de los controles de temperatura.
6. Observa el intercambio de calor en **tiempo real**.
7. Repite el experimento cuantas veces gustes.

¡Disfruta explorando el equilibrio térmico! 🔥❄️

In [26]:
# prompt: Can you design a gui with two blocks that change color according to an input like an scrollable bar
def hex_to_RGB(hex):
  ''' "#FFFFFF" -> [255,255,255] '''
  # Pass 16 to the integer function for change of base
  return [int(hex[i:i+2], 16) for i in range(1,6,2)]


def RGB_to_hex(RGB):
  ''' [255,255,255] -> "#FFFFFF" '''
  # Components need to be integers for hex to make sense
  RGB = [int(x) for x in RGB]
  return "#"+"".join(["0{0:x}".format(v) if v < 16 else
            "{0:x}".format(v) for v in RGB])


def color_dict(gradient):
  ''' Takes in a list of RGB sub-lists and returns dictionary of
    colors in RGB and hex form for use in a graphing function
    defined later on '''
  return {"hex":[RGB_to_hex(RGB) for RGB in gradient],
      "r":[RGB[0] for RGB in gradient],
      "g":[RGB[1] for RGB in gradient],
      "b":[RGB[2] for RGB in gradient]}


def linear_gradient(start_hex, finish_hex="#FFFFFF", n=10):
  ''' returns a gradient list of (n) colors between
    two hex colors. start_hex and finish_hex
    should be the full six-digit color string,
    inlcuding the number sign ("#FFFFFF") '''
  # Starting and ending colors in RGB form
  s = hex_to_RGB(start_hex)
  f = hex_to_RGB(finish_hex)
  # Initilize a list of the output colors with the starting color
  RGB_list = [s]
  # Calcuate a color at each evenly spaced value of t from 1 to n
  for t in range(1, n):
    # Interpolate RGB vector for color at the current value of t
    curr_vector = [
      int(s[j] + (float(t)/(n-1))*(f[j]-s[j]))
      for j in range(3)
    ]
    # Add it to our list of output colors
    RGB_list.append(curr_vector)

  return color_dict(RGB_list)

colores = linear_gradient( "#4682B4", "#FFB347", 255)
colores["g"][0]


import ipywidgets as widgets
from IPython.display import display
import time
import colour
import numpy as np

# Create two blocks (divs)
block1 = widgets.HTML(
    value='<div style="width:100px; height:100px; background-color:#4682B4;"></div>'
)
block2 = widgets.HTML(
    value='<div style="width:100px; height:100px; background-color:#4682B4;"></div>'
)

# Create a slider
slider1 = widgets.IntSlider(
    value=1,
    min=1,
    max=255,
    step=1,
    description='T1 [°C]:',
    continuous_update=True
)
slider2 = widgets.IntSlider(
    value=1,
    min=1,
    max=255,
    step=1,
    description='T2 [°C]:',
    continuous_update=True
)

sld_mass1 = widgets.IntSlider(
    value=100,
    min=50,
    max=200,
    step=1,
    description='Mass [g]:',
    continuous_update=True
)
sld_mass2 = widgets.IntSlider(
    value=100,
    min=50,
    max=200,
    step=1,
    description='Mass [g]:',
    continuous_update=True
)

# Create a play button
play_button = widgets.Button(description="▶ Play")

# Flag to track if animation is running
is_animating = False

#Function to calculate the temperature of equilibrium
def Temperature_eq(T1,T2,m1,m2):
    Teq = (T1*m1+T2*m2)/(m1+m2)
    print(f"{Teq:.2f}",T1,T2)
    r = colores["r"][int(np.round(Teq))]
    g = colores["g"][int(np.round(Teq))]
    b = colores["b"][int(np.round(Teq))]
    return np.round(Teq), r, g, b


# Function to disable/enable GUI elements
def disable_gui(disable):
    play_button.disabled = disable
    slider1.disabled = disable
    slider2.disabled = disable
    sld_mass1.disabled = disable
    sld_mass2.disabled = disable

# Function to update the block colors
def update_colors_sizes(change):

  intensity1 = slider1.value
  intensity2 = slider2.value

  size1 = sld_mass1.value
  size2 = sld_mass2.value

  # Update block1 with gradient color
  block1.value = f'<div style="width:{size1}px; height:{size1}px; background-color:rgb({colores["r"][intensity1-1]}, {colores["g"][intensity1-1]}, {colores["b"][intensity1-1]});"></div>'
  block2.value = f'<div style="width:{size2}px; height:{size2}px; background-color:rgb({colores["r"][intensity2-1]}, {colores["g"][intensity2-1]}, {colores["b"][intensity2-1]});"></div>'

# Function to update the block colors (animation)
def animate_colors():
    global is_animating

    if is_animating:  # Check if animation is already running
        return

    is_animating = True
    disable_gui(True)  # Disable GUI

    try:
        T1 = slider1.value
        T2 = slider2.value
        m1 = sld_mass1.value
        m2 = sld_mass2.value

        T_eq, r_eq,g_eq,b_eq = Temperature_eq(T1,T2,m1,m2)

        steps = np.max((np.abs(T1-T_eq),np.abs(T2-T_eq)))
        step_1 = np.abs(T1-T_eq)/steps
        step_2 = np.abs(T2-T_eq)/steps

        for i in range(1,int(steps)+1):

          if T1 < T2:
            r1 = colores["r"][ T1 + int(np.round(i*step_1)) ]
            r2 = colores["r"][ T2 - int(np.round(i*step_2)) ]
            g1 = colores["g"][ T1 + int(np.round(i*step_1)) ]
            g2 = colores["g"][ T2 - int(np.round(i*step_2)) ]
            b1 = colores["b"][ T1 + int(np.round(i*step_1)) ]
            b2 = colores["b"][ T2 - int(np.round(i*step_2)) ]

            # Update the slider values
            slider1.value = T1 + int(np.round(i*step_1))
            slider2.value = T2 - int(np.round(i*step_2))

            # Update block1 with gradient color
            block1.value = f'<div style="width:{m1}px; height:{m1}px; background-color:rgb({r1}, {g1}, {b1});"></div>'
            block2.value = f'<div style="width:{m2}px; height:{m2}px; background-color:rgb({r2}, {g2}, {b2});"></div>'
            time.sleep(0.05)  # Adjust animation speed here

          else:
            r1 = colores["r"][ T1 - int(np.round(i*step_1)) ]
            r2 = colores["r"][ T2 + int(np.round(i*step_2)) ]
            g1 = colores["g"][ T1 - int(np.round(i*step_1)) ]
            g2 = colores["g"][ T2 + int(np.round(i*step_2)) ]
            b1 = colores["b"][ T1 - int(np.round(i*step_1)) ]
            b2 = colores["b"][ T2 + int(np.round(i*step_2)) ]

            # Update the slider values
            slider1.value = T1 - int(np.round(i*step_1))
            slider2.value = T2 + int(np.round(i*step_2))

            # Update block1 with gradient color
            block1.value = f'<div style="width:{m1}px; height:{m1}px; background-color:rgb({r1}, {g1}, {b1});"></div>'
            block2.value = f'<div style="width:{m2}px; height:{m2}px; background-color:rgb({r2}, {g2}, {b2});"></div>'
            time.sleep(0.05)  # Adjust animation speed here

    finally:
        is_animating = False
        disable_gui(False)  # Re-enable GUI




# Observe the slider for changes
slider1.observe(update_colors_sizes, names='value')
slider2.observe(update_colors_sizes, names='value')
sld_mass1.observe(update_colors_sizes, names='value')
sld_mass2.observe(update_colors_sizes, names='value')

# Function to handle button click
def on_play_button_clicked(b):
    animate_colors()  # Start animation

# Observe the button click event
play_button.on_click(on_play_button_clicked)

# HBox for side-by-side layout
row = widgets.HBox(
    [block1, block2],
    layout=widgets.Layout(
        #justify_content='center',  # Centers horizontally
        align_items='center',  # Aligns blocks in the middle of HBox
        spacing=10  # Adds some space between blocks
    )
)

# VBox to center the row vertically
container = widgets.VBox(
    [row],
    layout=widgets.Layout(
        align_items='center',  # Center contents vertically
        #justify_content='center',  # Center contents horizontally
        height='60vh',  # Adjust height so it doesn’t take too much space
        padding='10px'
    )
)
display(container)
display(slider1)
display(slider2)
display(sld_mass1)
display(sld_mass2)
display(play_button)

IntSlider(value=1, description='T1:', max=255, min=1)

IntSlider(value=1, description='T2:', max=255, min=1)

IntSlider(value=100, description='Mass:', max=200, min=50)

IntSlider(value=100, description='Mass:', max=200, min=50)

Button(description='▶ Play', style=ButtonStyle())